In [47]:
# stacked generalization with linear meta model on blobs dataset
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack
import tensorflow as tf
import numpy as np

In [48]:
# load models from file
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = './models/imageclassifier_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [49]:
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX

In [50]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = LogisticRegression()
	print(np.shape(stackedX))
	print(np.shape(inputy))
	model.fit(stackedX, inputy)
	return model

In [51]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	print(np.shape(stackedX))
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [52]:
numbers1 = [1, 2, 3]
numbers2 = [4, 5, 6]
result = map(lambda x, y: (x/255, y), numbers1,numbers2)
print(list(result))

[(0.00392156862745098, 4), (0.00784313725490196, 5), (0.011764705882352941, 6)]


In [53]:
data = tf.keras.utils.image_dataset_from_directory('Images')
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()
data = data.map(lambda x,y: (x/255, y))
train_size = int(len(data)*.6)
val_size = int(len(data)*.2)
test_size = int(len(data)*.2)
train = data.take(train_size)
print(np.shape(train))
print(list(train))
trainy = list(train.unbatch().map(lambda x, y: y))
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
testy = list(test.unbatch().map(lambda x, y: y))

Found 202 files belonging to 2 classes.
()
[(<tf.Tensor: shape=(32, 256, 256, 3), dtype=float32, numpy=
array([[[[0.9757966 , 0.9757966 , 1.        ],
         [0.96660537, 0.96660537, 1.        ],
         [0.9607843 , 0.9607843 , 0.9966299 ],
         ...,
         [0.6576899 , 0.6851409 , 0.7871017 ],
         [0.5405024 , 0.5679534 , 0.6699142 ],
         [0.8851716 , 0.9126226 , 0.9679841 ]],

        [[0.9562653 , 0.9562653 , 0.9991726 ],
         [0.94331324, 0.94331324, 0.9862353 ],
         [0.9351103 , 0.9351103 , 0.9709559 ],
         ...,
         [0.5443632 , 0.5718142 , 0.673775  ],
         [0.5841282 , 0.6115792 , 0.71354   ],
         [0.7958139 , 0.8232649 , 0.9167614 ]],

        [[0.94349205, 0.9489608 , 0.9960196 ],
         [0.92720586, 0.9333796 , 0.9777114 ],
         [0.91847426, 0.9266774 , 0.9597886 ],
         ...,
         [0.440639  , 0.47082436, 0.564582  ],
         [0.6540957 , 0.68428105, 0.77803874],
         [0.6971962 , 0.7273816 , 0.8211392 ]],

  

In [58]:
# load all models
n_members = 2
members = load_all_models(n_members)
print('Loaded %d models' % len(members))
# evaluate standalone models on test dataset
for model in members:
	_, acc = model.evaluate(train, verbose=0)
	print('Model Accuracy: %.3f' % acc)

>loaded ./models/imageclassifier_1.h5
>loaded ./models/imageclassifier_2.h5
Loaded 2 models
Model Accuracy: 0.547
Model Accuracy: 0.656


In [55]:
model = fit_stacked_model(members, train, trainy)

(128, 2)
(128,)


In [56]:
yhat = stacked_prediction(members, model, train)

(128, 2)


In [57]:
acc = accuracy_score(trainy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)

Stacked Test Accuracy: 0.445
